In [2]:
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages')
sys.path.append('/Users/JP/nltk_data')
import requests, re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from io import StringIO
from nsepy import get_history
import glob
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup as bs
import requests 
import numpy as np
import pandas as pd
import re #Using regular expressions to search for special sequence of characters
# import winsound
import time

/Users/JP/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/JP/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [28]:
IndustryLinks=np.empty(0)
page=requests.get("https://www.moneycontrol.com/stocks/marketstats/industry-classification/nse/abrasives.html")
soup=bs(page.content,'lxml') #creating soup object to access links of all the industries

attr={'class':'colLft'}
section=soup.find('section',attrs=attr)#Scanning for section that contains list of all links


attr2={'href':re.compile("/stocks/marketstats/industry-classification/nse/")}
links=section.find_all('a',attrs=attr2)#storing all href containing links after parsing isdone

IndustryLinks=np.append(IndustryLinks,'https://www.moneycontrol.com/stocks/marketstats/industry-classification/nse/abrasives.html')
link1='https://www.moneycontrol.com'

In [29]:
for link in links:
    IndustryLinks=np.append(IndustryLinks,link1+str(link.get('href')))
#Extracting all industry links from html doc and storing them in numpy array
industry_names=['Abrasives']
for i in links :
    industry_names.append(i.text)
industries=dict(zip(industry_names,IndustryLinks))


191


In [96]:
# c=0
# company_classification= pd.DataFrame()
# for i in industries:
#     link=industries[i]
#     page=requests.get(link)
#     soup=bs(page.content,'lxml')
#     attr={'border':'0','cellpadding':'0','cellspacing':'0','width':'100%'}
#     table=soup.find('table',attrs=attr)
#     tbody=table.find('tbody')
#     Complinks=tbody.find_all('a',attrs={'href':re.compile("https://www.moneycontrol.com/india/stockpricequote/")})
#     for j in Complinks:
#         company_classification.loc[c,'link']=j.attrs['href']
#         company_classification.loc[c,'Name']=j.text
#         company_classification.loc[c,'Industry']=i
#         c+=1


In [568]:
def fetch_terms(Clink):
    temp=[]
    page=requests.get(Clink)
    soup=bs(page.content,'lxml')
    attr={'class':'valuation_section','id':'sec_valul'}
    div=soup.find('section',attrs=attr)
    s=div.div.div.find('div',{'class':'valuation_lft'}).div.find('div',{'class':'tab-content MT10'}).find('div',{'class':"tab-pane fade " ,'id':"consolidated_valuation" ,'role':'tabpanel'}).ul
    terms = {}
    for x in s.contents:
        if x!='\n':
            for i in x.findAll('li'):
                if i.findAll('div')[0].text!='Deliverables (%)':
                    terms[i.findAll('div')[0].text]=(i.findAll('div')[1].text)    
                else:
                    for a in x.findAll('div',{'class':'divTableRow'}):
                        terms[a.findAll('div')[0].text]=a.findAll('div')[1].text
                    terms['Deliverable%']=x.find('a').text
    return terms

In [569]:
def fetch_beta(Clink):    
    page=requests.get(Clink)       #Using companylink passed tocreate soup object
    soup=bs(page.content,'lxml')
    attr4={'class':'nsert','id':'div_nse_livebox_wrap'}           #Using attributes to locate nse section of 
    nseData=soup.find('div',attrs=attr4)                          #html doc
    beta=nseData.find_all('div',attrs={'class':'disin vt'})       #Finding the part of the code 
    temp=beta[1].text.split('Beta\n')[1].split('\n\n')[0]
    return np.float(temp)

In [570]:
final_result=pd.DataFrame()

In [573]:
for i in range(len(company_classification)):
    try:
        l=company_classification.loc[i].link
        beta=fetch_beta(l)
        values= fetch_terms(l)
        name=company_classification.loc[i].Name
        results=pd.Series(values)
        results['Beta']=beta
        results['Industry']=company_classification.loc[i].Industry
        final_result[name]=results
        print(i*100/len(company_classification))
    except:
        print("Error in ",company_classification.loc[i].Name)

0.0
0.06289308176100629
0.12578616352201258
0.18867924528301888
0.25157232704402516
Error in  Bombay Super
0.37735849056603776
0.44025157232704404
0.5031446540880503
0.5660377358490566
0.6289308176100629
0.6918238993710691
Error in  ICE Make Refrig
0.8176100628930818
0.8805031446540881
0.9433962264150944
1.0062893081761006
1.069182389937107
1.1320754716981132
1.1949685534591195
1.2578616352201257
1.320754716981132
1.3836477987421383
1.4465408805031446
1.509433962264151
1.5723270440251573
1.6352201257861636
1.6981132075471699
1.7610062893081762
1.8238993710691824
1.8867924528301887
1.949685534591195
2.0125786163522013
2.0754716981132075
2.138364779874214
2.20125786163522
2.2641509433962264
2.3270440251572326
2.389937106918239
2.452830188679245
2.5157232704402515
2.5786163522012577
2.641509433962264
2.7044025157232703
2.7672955974842766
2.830188679245283
2.893081761006289
2.9559748427672954
3.018867924528302
3.0817610062893084
3.1446540880503147
3.207547169811321
3.270440251572327
3.3333

In [567]:
company_classification.to_csv('/Users/JP/Desktop/links_of_companies.csv')

In [574]:
final_result.to_csv('/Users/JP/Desktop/results.csv')

In [583]:
final_result.isna().sum(axis=1)

Market Cap (Rs Cr.)    0
P/E                    0
Book Value (Rs)        0
Dividend (%)           0
Market Lot             0
Industry P/E           0
EPS (TTM)              0
P/C                    0
Price/Book             0
Dividend Yield.(%)     0
Face Value (RS)        0
3-Day                  0
5-Day                  0
8-Day                  0
Deliverable%           0
Beta                   0
Industry               0
dtype: int64

In [584]:
final_result

,Grindwell Norto,Wendt,Hindustan Aeron,Agri-Tech,Bombay Burmah,Dhunseri Tea,Harrisons Malay,Kaveri Seed,Nath Bio-Genes,Amber Enterpris,...,JVL Agro Ind,NK Industries,Ruchi Soya,Ruchinfra,Sanwaria Consum,KDDL,Archidply Ind,CenturyPlyboard,Greenply Ind,Mangalam Timber
Market Cap (Rs Cr.),"5,054.37",377.00,"17,943.23",11.76,"5,488.96",86.86,101.50,"1,849.69",283.73,"3,744.18",...,5.21,12.50,"5,844.34",43.92,86.86,126.11,38.50,"2,336.15","1,077.89",13.93
P/E,27.85,34.74,6.33,-,5.05,0.89,14.69,7.79,-,23.17,...,-,-,151.98,-,-,24.42,-,13.80,10.87,-
Book Value (Rs),100.41,651.29,324.78,-,989.96,1081.60,42.56,166.70,-,319.63,...,-22.82,-814.65,-169.99,6.35,8.87,200.84,-,45.02,27.34,-
Dividend (%),120.00,300.00,198.00,0.00,50.00,50.00,0.00,150.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,25.00,0.00,100.00,40.00,0.00
Market Lot,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Industry P/E,27.11,27.11,9.06,13.74,43.59,43.59,29.71,13.74,13.74,17.05,...,13.97,13.97,13.97,13.97,13.97,13.14,12.72,12.43,12.72,12.72
EPS (TTM),16.57,54.05,84.41,-,156.08,138.44,3.77,39.84,-,51.36,...,-,-,1.31,-,-,4.51,-,7.59,8.03,-
P/C,22.10,17.88,4.38,-,4.35,0.75,9.12,7.02,-,16.73,...,-,2.47,32.98,5.10,1.44,6.86,-,10.24,8.91,-
Price/Book,4.55,2.89,1.65,-,0.79,0.11,1.29,1.84,-,3.73,...,-,-,-,0.34,0.13,0.54,-,2.34,3.22,-
Dividend Yield.(%),1.31,1.59,3.69,-,0.13,4.03,-,0.98,-,-,...,-,-,-,-,-,2.31,-,0.95,0.46,-


In [550]:
final_result

,chutya
Market Cap (Rs Cr.),"5,488.96"
P/E,5.05
Book Value (Rs),989.96
Dividend (%),50.00
Market Lot,1
Industry P/E,43.59
EPS (TTM),156.08
P/C,4.35
Price/Book,0.79
Dividend Yield.(%),0.13
